<a href="https://colab.research.google.com/github/trighpro123/Project_chatbot/blob/main/Model_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim
!pip install tensorflow_addons
!pip install vncorenlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 27.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 22.8 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=a6dc4099ae3f2fa7ece8c585b6173bd461f56876cc000cf8755a1f35b03bcaaf
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
from sklearn.metrics import *
import os, pickle, re, keras, sklearn, string
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
import io
import numpy as np
from numpy import random
import random as rn
random.seed(42)
import tensorflow as tf
np.random.seed(42)
rn.seed(42)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!git clone https://github.com/vncorenlp/VnCoreNLP
from vncorenlp import VnCoreNLP
import re
# replace path_vncorenlp here
path_vncorenlp = "/content/VnCoreNLP/VnCoreNLP-1.1.1.jar"
rdrsegmenter = VnCoreNLP(path_vncorenlp, annotators="wseg", max_heap_size='-Xmx500m') 

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.


In [ ]:
import string
import re

def clean_doc(text, word_segment = True):
  
  text = text.lower()
  #xóa dấu câu
  for punc in string.punctuation:
    text = text.replace(punc,' ')
  #xóa khoảng trắng thừa
  text = re.sub('\s+',' ', text)
  text = text.strip()
  #tách từ
  if word_segment == True:
    text = rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
  else:
    pass
  return text
text = '   xét tuyển như thế nào?!   '
print(clean_doc(text))

xét tuyển như_thế_nào


In [ ]:
import pandas as pd
#thay đường dẫn data
path_data = '/content/drive/MyDrive/AI/Data_chatbot/data.xlsx'
path_ans = '/content/drive/MyDrive/AI/Data_chatbot/Answer.xlsx'
data = pd.read_excel(path_data)
answers_df = pd.read_excel(path_ans)
ans = answers_df['Answer'].tolist()

data

,Question,Answer,Class
0,Hồ sơ đăng ký nhập học tại trường gồm những gì,Hồ sơ bao gồm: bằng tốt nghiệp hoặc giấy chứng...,1.0
1,Hồ sơ nhập học trường UTE như thế nào,Hồ sơ bao gồm: bằng tốt nghiệp hoặc giấy chứng...,1.0
2,Đi đăng ký nhập học cần đem những gì,Hồ sơ bao gồm: bằng tốt nghiệp hoặc giấy chứng...,1.0
3,Nhập học cần đem hồ sơ như thế nào,Hồ sơ bao gồm: bằng tốt nghiệp hoặc giấy chứng...,1.0
4,Muốn nhập học cần những loại giấy tờ gì,Hồ sơ bao gồm: bằng tốt nghiệp hoặc giấy chứng...,1.0
...,...,...,...
307,"Trong môi trường Nhà trường, các bạn nữ sẽ đượ...",Nhà trường cấp học bổng bằng 50% học phí toàn ...,31.0
308,Nhà trường có ưu ái gì cho sinh viên nữ không?,Nhà trường cấp học bổng bằng 50% học phí toàn ...,31.0
309,Nhà trường có chính sách đặc biệt nào đối với ...,Nhà trường cấp học bổng bằng 50% học phí toàn ...,31.0
310,Sinh viên nữ được ưu tiên gì ở trường?,Nhà trường cấp học bổng bằng 50% học phí toàn ...,31.0


In [ ]:
### Chuyển số dạng string sang dạng int
def standar_label(label):
  return int(label)
category_list = data['Class'].apply(standar_label).unique().tolist() ### Tìm các phần tử duy nhất trong mảng
print(category_list)
### Vecto hóa số thứ tự các câu hỏi
def to_category_vector(category, target_categories = category_list):
    vector = np.zeros(len(target_categories)).astype(np.float32)
    for i in range(len(target_categories)):
        if target_categories[i] == category:
            vector[i] = 1.0
            break
    return vector
print(to_category_vector(3))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
## Gán và xử lý dữ liệu
questions = data['Question'].astype(str)
questions = questions.apply(clean_doc).tolist()
Y_train = data['Class'].apply(to_category_vector).tolist()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer(oov_token='<UNK>')
# cập nhập từ vựng nội bộ dựa trên questions trước khi biến đổi sequences
t.fit_on_texts(questions)
t.word_index['<PAD>'] = 0
# Với mỗi từ sẽ gán với 1 số tương ứng

In [ ]:
input_vocab_size = len(t.word_index)
print(input_vocab_size)
MAX_LEN = 40
EMBEDDING_DIM = 300

328


In [ ]:
### Chuyển chuỗi văn bản questions sang chuỗi số
train_sequences = t.texts_to_sequences(questions)
# Chèn các số từ câu biến thành vào vecto có độ dài 40, các số rỗng sẽ thành 0 
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_LEN)
X_train

array([[  0,   0,   0, ..., 163,  36,  14],
       [  0,   0,   0, ...,  13,  12,   5],
       [  0,   0,   0, ..., 139,  36,  14],
       ...,
       [  0,   0,   0, ...,  41,  89,   6],
       [  0,   0,   0, ...,  14,  54,   2],
       [  0,   0,   0, ..., 189,  36,  14]], dtype=int32)

In [ ]:
import tensorflow_addons as tfa
filter_nums = 64

inputs  = tf.keras.layers.Input(shape=(MAX_LEN, ), dtype='float64')    #Tạo cấu trúc input cho CNN   
# Scale lại dữ liệu thành các vecto/ số fit vs model
# 1 size của từ vựng input; 2 size dense output
embedding_layer_domain = tf.keras.layers.Embedding(input_vocab_size, EMBEDDING_DIM, input_length=MAX_LEN)(inputs) 
embedding_layer_domain = tf.keras.layers.SpatialDropout1D(0.5)(embedding_layer_domain) # hàm của dropout

# Xây dựng model
conv_1 = tf.keras.layers.Conv1D(filter_nums, 2, padding="same", activation="relu",
                                kernel_initializer='he_normal',trainable=True)(embedding_layer_domain)        
conv_2 = tf.keras.layers.Conv1D(filter_nums, 3, padding="same", activation="relu",
                                kernel_initializer='he_normal',trainable=True)(embedding_layer_domain)
conv_1 = tf.keras.layers.Dropout(0.2)(conv_1)
conv_2 = tf.keras.layers.Dropout(0.2)(conv_2)
maxpool_1 = tf.keras.layers.GlobalMaxPooling1D()(conv_1)
maxpool_2 = tf.keras.layers.GlobalMaxPooling1D()(conv_2)

v0_col = tf.keras.layers.Concatenate(axis=1)([maxpool_1, maxpool_2])
dense_1 = tf.keras.layers.Dense(300,name="dense1", activation='relu')(v0_col)
drop = tf.keras.layers.Dropout(0.2)(dense_1)
output = tf.keras.layers.Dense(len(category_list),name="output1", activation='softmax')(drop)

# Training CNN
model = tf.keras.Model(inputs=inputs, outputs=output)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(loss=loss, optimizer= tf.keras.optimizers.Adam(0.001) , metrics = "accuracy")
history = model.fit(X_train, np.array(Y_train), batch_size=64, epochs=50)
#model.load_weights(checkpoint_filepath)

Epoch 1/50
5/5 [==============================] - 13s 68ms/step - loss: 3.4414 - accuracy: 0.0256
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 3.3931 - accuracy: 0.0929
Epoch 3/50
5/5 [==============================] - 0s 8ms/step - loss: 3.3569 - accuracy: 0.1891
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 3.3013 - accuracy: 0.3077
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 3.2241 - accuracy: 0.4327
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 3.1222 - accuracy: 0.5224
Epoch 7/50
5/5 [==============================] - 0s 5ms/step - loss: 2.9674 - accuracy: 0.6346
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 2.7623 - accuracy: 0.7532
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 2.5114 - accuracy: 0.7853
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 2.1802 - accuracy: 0.8622
Epoch 11/50
5/5 [====================

In [ ]:
def predict_class(text1):
  test_encoded1 = t.texts_to_sequences([clean_doc(text1)])
  test_pad1 = sequence.pad_sequences(test_encoded1, maxlen=MAX_LEN)
  question1_class = np.argmax(model.predict(test_pad1))
  return question1_class

In [ ]:
## Test sơ bộ
ans[predict_class('Chương trình sư phạm và chương trình công nghệ khác nhau như thế nào?')]

'Khác nhau:\n_ Chương trình Sư phạm học 4,5 năm; có học khối kiến thức sư phạm (tương đương khoảng 01 học kỳ); trong quá trình học, SV được miễn học phí; khi tốt nghiệp, SV được cấp 02 văn bằng: Bằng kỹ sư (hoặc cử nhân) theo ngành học & chứng chỉ sư phạm kỹ thuật; SV phải chấp hành sự phân công công tác trong ngành giáo dục theo quy định, nếu Nhà nước có yêu cầu.\n_ Chương trình công nghệ học 4 năm; không học khối kiến thức sư phạm; trong quá trình học SV phải đóng học phí; khi tốt nghiệp, SV được cấp 01 văn bằng: Bằng kỹ sư (hoặc cử nhân) theo ngành học.\nGiống nhau: \n_ Đều là hệ đại học chính quy; khối kiến thức đại cương, cơ sở ngành, chuyên ngành học giống nhau.'

In [ ]:
model.save('model_chatbot.h5')